# Tools

There are 3 types of Tools :

1. Custom Functions
2. Built In
3. 3rd Party


# 🔧 Using Tools in AutoGen Agent Framework

AutoGen supports **tool-augmented agents** to enable LLMs to call external functions, APIs, or internal utilities. These tools fall into three primary categories:

---

## 1️⃣ Custom Functions (User-Defined Tools)

AutoGen allows you to define your own Python functions and expose them to LLMs as tools using `FunctionTool`.

### ✅ Example: Reversing a String

```python
from autogen_core.tools import FunctionTool

def reverse_string(text: str) -> str:
    return text[::-1]

reverse_tool = FunctionTool(
    reverse_string,
    description="A tool to reverse a string"
)
```

Register the tool with an assistant agent:

```python
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name="ReverseStringAgent",
    model_client=model_client,
    tools=[reverse_tool],
    system_message="You can reverse strings using the tool."
)
```

---

## 2️⃣ Built-In Tools

AutoGen internally supports some built-in capabilities such as:

- `ToolNode` for conditionally invoking tools
- `StateGraph` for memory/state-aware agent routing
- System messages for reasoning control

These built-ins can be used to build **multi-agent workflows**, conditional logic flows, and memory-based interactions.

---

## 3️⃣ Third-Party APIs or External Services

You can also wrap external services or APIs and expose them as callable tools using the same `FunctionTool` interface.

### ✅ Example: Calling a Weather API

```python
import requests
from autogen_core.tools import FunctionTool

def get_weather(city: str) -> str:
    api_key = "your_api_key"
    response = requests.get(f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={city}")
    return response.json()["current"]["condition"]["text"]

weather_tool = FunctionTool(get_weather, description="Get current weather for a city")
```

Add it to the agent:

```python
agent = AssistantAgent(
    name="WeatherAgent",
    model_client=model_client,
    tools=[weather_tool],
    system_message="Use the weather tool to answer weather-related queries."
)
```

---

## 🧠 Summary Table

| Type             | Description                                  | AutoGen Component                |
|------------------|----------------------------------------------|----------------------------------|
| Custom Functions | Python logic defined and exposed via tool    | `FunctionTool`                   |
| Built-In         | Internal routing, memory, system reasoning   | `ToolNode`, `StateGraph`, etc.   |
| 3rd Party        | APIs or SDKs wrapped as callable tools       | `FunctionTool` with HTTP/SDK     |

---

## 📝 Final Tips

- Use `FunctionTool` to expose any callable Python function to the LLM.
- Provide a helpful `description` so the LLM knows **when** and **how** to use it.
- AutoGen handles tool invocation and type parsing automatically at runtime.

---

Happy building with AutoGen Tools! ⚙️


In [ ]:
import asyncio
from autogen_agentchat.agents import AssistantAgent  # To create an assistant agent
from autogen_ext.models.openai import OpenAIChatCompletionClient  # OpenAI client wrapper
from autogen_core.tools import FunctionTool  # To define and register tools
import os
from dotenv import load_dotenv  # For loading API keys from a .env file

# Load environment variables from .env file
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

# Initialize the OpenAI LLM client with GPT-4o
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

# Define a basic Python function to reverse a string
def reverse_string(text: str) -> str:
    '''
    Reverse the given text.

    Parameters:
    - text (str): Input string

    Returns:
    - str: Reversed string
    '''
    return text[::-1]

# Wrap the reverse_string function as a tool for the agent
reverse_tool = FunctionTool(
    reverse_string,
    description='A tool to reverse a string'
)

# Create an assistant agent and register the reverse_string tool
agent = AssistantAgent(
    name="ReverseStringAgent",
    model_client=model_client,
    system_message='You are a helpful assistant that can reverse strings using the reverse_string tool. Give the result with a brief summary.',
    tools=[reverse_tool]
)

# Run the agent on a task and await the result
result = await agent.run(task='Reverse the string "Hello, World!"')

# Output the result
print(result)
